# Rigid Body Dynamics and Simulation

Finally, let's see how to start assembling the tools mentioned in the Systems and Symbolic/Autodiff tutorials to make robots do interesting things.

For these examples, we'll explore simulating the ultra-classic cart pole, pictured below.

<img src="https://danielpiedrahita.files.wordpress.com/2017/02/cart-pole.png" alt="drawing" style="width: 400px;"/>

In [14]:
import math
import matplotlib.pyplot as plt
import numpy as np

from pydrake.all import (BasicVector, DiagramBuilder, FloatingBaseType,
                         LinearQuadraticRegulator, RigidBodyPlant,
                         RigidBodyTree, Simulator, SignalLogger)
from underactuated import (PlanarRigidBodyVisualizer)
from IPython.display import HTML

# Load in the cartpole from its URDF
tree = RigidBodyTree("cartpole/cartpole.urdf",
                     FloatingBaseType.kFixed)

# Define an upright state
def UprightState():
    state = (0, math.pi, 0, 0)
    return state

print "nq: ", tree.get_num_positions()
print "nv: ", tree.get_num_velocities()

nq:  2
nv:  2


That RigidBodyTree keeps track of the kinematics and dynamics (minus contact interactions) of the robot. You can use it to, for example, calculate forward kinematics:

In [27]:
kinsol = tree.doKinematics(UprightState())
world_body_index = tree.world().get_body_index()
target_body_index = tree.FindBody("pole").get_body_index()
end_of_pole_in_world_frame = tree.transformPoints(kinsol, [0., 0., -0.5], target_body_index, world_body_index)
print end_of_pole_in_world_frame

[[-6.123234e-17]
 [ 0.000000e+00]
 [ 7.500000e-01]]


As a more complete demo, we can create an LQR solution around that upright fixed point and simulate it!

See the quickstart guide for a written explanation of the many pieces of this.

In [10]:
def BalancingLQR(robot):
    # Design an LQR controller for stabilizing the CartPole around the upright.
    # Returns a (static) AffineSystem that implements the controller (in
    # the original CartPole coordinates).

    context = robot.CreateDefaultContext()
    context.FixInputPort(0, BasicVector([0]))

    context.get_mutable_continuous_state_vector().SetFromVector(UprightState())

    Q = np.diag((10., 10., 1., 1.))
    R = [1]

    return LinearQuadraticRegulator(robot, context, Q, R)


builder = DiagramBuilder()

robot = builder.AddSystem(RigidBodyPlant(tree))
controller = builder.AddSystem(BalancingLQR(robot))
builder.Connect(robot.get_output_port(0), controller.get_input_port(0))
builder.Connect(controller.get_output_port(0), robot.get_input_port(0))

logger = builder.AddSystem(SignalLogger(robot.get_output_port(0).size()))
logger._DeclarePeriodicPublish(1. / 30., 0.0)
builder.Connect(robot.get_output_port(0), logger.get_input_port(0))

diagram = builder.Build()
simulator = Simulator(diagram)
simulator.set_publish_every_time_step(False)
context = simulator.get_mutable_context()

state = context.get_mutable_continuous_state_vector()
state.SetFromVector(UprightState() + 0.1*np.random.randn(4,))
simulator.StepTo(10.)

prbv = PlanarRigidBodyVisualizer(tree, xlim=[-2.5, 2.5], ylim=[-1, 2.5])
ani = prbv.animate(logger, resample=30, repeat=True)
plt.close(prbv.fig)
HTML(ani.to_html5_video())

Spawning PlanarRigidBodyVisualizer for tree with 1 actuators
